# [構建一個簡單的 LLM 應用程序：使用 LCEL](https://python.langchain.com/v0.2/docs/tutorials/llm_chain/)
========================

在這個快速入門指南中，將展示如何使用 LangChain 構建一個簡單的 LLM 應用程式。這個應用程式將把文本從英文翻譯成其他語言。雖然這是一個相對簡單的 LLM 應用程序------只涉及一次 LLM 呼叫和一些提示語，但這是一個很好的起點。通過簡單的提示語和 LLM 調用，你可以構建出許多強大的功能！

閱讀完本教程後，你將對以下內容有一個概略的了解：

-   如何使用 [語言模型](https://python.langchain.com/v0.2/docs/concepts/#chat-models)
-   如何使用 [PromptTemplates](https://python.langchain.com/v0.2/docs/concepts/#prompt-templates) 和 [OutputParsers](https://python.langchain.com/v0.2/docs/concepts/#output-parsers)
-   如何使用 [LangChain Expression Language (LCEL)](https://python.langchain.com/v0.2/docs/concepts/#langchain-expression-language-lcel) 來串聯組件
-   如何使用 [LangSmith](https://python.langchain.com/v0.2/docs/concepts/#langsmith) 進行應用程序的調試和追踪
-   如何使用 [LangServe](https://python.langchain.com/v0.2/docs/concepts/#langserve) 來部署你的應用程序

讓我們開始吧！

In [ ]:
# ! pip3 install langchain

許多使用 LangChain 開發的應用程序都包含多個步驟，其中涉及多次 LLM 調用。隨著這些應用程序變得越來越複雜，能夠檢查鏈或代理內部發生的情況變得至關重要。而檢查這些過程的最佳方法就是使用 [LangSmith](https://smith.langchain.com/)。

In [ ]:
# export LANGCHAIN_TRACING_V2="true"
# export LANGCHAIN_API_KEY="..."

我們採用 [Ollama](https://ollama.com/) 離線模型需要在本地安裝後，使用指令下載模型，本篇預設採用 "llama3.1"

In [ ]:
# ! ollama pull llama3.1

接著我們就可以使用 langchain_ollama 來使用模型

In [1]:
# import os
# from langchain_openai import AzureChatOpenAI
# model = AzureChatOpenAI(
#     azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
#     azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
#     openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
# )

from langchain_ollama import ChatOllama
# model_name = "jcai/taide-lx-7b-chat"
model_name = "llama3.1"
model = ChatOllama(model=model_name)
model

ChatOllama(model='jcai/taide-lx-7b-chat')

首先，我們來直接使用模型。ChatOllama 是 LangChain 中的 "Runnables" 實例，這意味著它們提供了一個標準的接口來與它們進行交互。要簡單地調用模型，我們可以將一組消息作為參數傳遞給 `.invoke` 方法。

In [2]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="請將英文翻譯成中文"),
    HumanMessage(content="hi!"),
]
result = model.invoke(messages)
result.content

'嗨！'

### OutputParsers
請注意，模型的回應是一個 AIMessage，它包含了一個字串回應以及其他相關的元數據。通常，我們可能只想處理字串回應。我們可以使用一個簡單的輸出解析器來解析出這個字串回應。

首先，我們需要導入這個簡單的輸出解析器。

In [4]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
parser.invoke(result)

'你好！'

更常見的是，我們可以將模型與這個輸出解析器 `Chain (鏈)` 起來。這意味著每次在這個鏈中，輸出解析器都會被調用。這個鏈將採用語言模型的輸入類型（字串或消息列表），並返回輸出解析器的輸出類型（字串）。

我們可以輕鬆地使用 `|` 運算符來創建這個鏈。在 LangChain 中，`|` 運算符用於將兩個元素結合在一起。

In [6]:
chain = model | parser

In [11]:
chain.invoke(messages)

'嗨！'

如果我們現在查看 LangSmith，我們可以看到這個鏈有兩個步驟：首先調用語言模型，然後將結果傳遞給輸出解析器。我們可以在 [LangSmith trace](https://smith.langchain.com/public/f1bdf656-2739-42f7-ac7f-0f1dd712322f/r) 中查看這個過程。

[Prompt Templates](https://python.langchain.com/v0.2/docs/tutorials/llm_chain/#prompt-templates "Direct link to Prompt Templates")

-----------------------------------------------------------------------------------------------------------------------------------

目前，我們直接將一組消息列表傳遞給語言模型。那麼這些消息列表從哪裡來呢？通常，這些列表是由用戶輸入和應用邏輯的結合所構建的。應用邏輯通常會將原始用戶輸入轉換為一個準備好傳遞給語言模型的消息列表。常見的轉換包括添加系統消息或使用用戶輸入來格式化模板。

提示模板 (PromptTemplates) 是 LangChain 中的一個概念，專為幫助這種轉換而設計。它們接收原始的用戶輸入，並返回準備好傳遞給語言模型的數據（即提示）。

現在，我們來創建一個提示模板。它將接收兩個用戶變量：

-   `language`: 要翻譯的目標語言

-   `text`: 要翻譯的文本

In [8]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "將內容翻譯成 {language}:"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

result = prompt_template.invoke({"language": "中文", "text": "hi"})

result

ChatPromptValue(messages=[SystemMessage(content='將內容翻譯成 中文:'), HumanMessage(content='hi')])

我們可以看到，它返回了一個包含兩條消息的 `ChatPromptValue`。如果我們想直接訪問這些消息，我們可以這樣做：

In [9]:
result.to_messages()

[SystemMessage(content='將內容翻譯成 中文:'), HumanMessage(content='hi')]

[將 components 使用 LCEL Chaining 起來](https://python.langchain.com/v0.2/docs/tutorials/llm_chain/#chaining-together-components-with-lcel "Direct link to Chaining together components with LCEL")

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

我們現在可以使用管道 (`|`) 運算符將這個提示模板與之前的模型和輸出解析器結合起來：

In [10]:
chain = prompt_template | model | parser

In [11]:
chain.invoke({"language": "中文", "text": "hi"})

'你好！'

這是一個使用 [LangChain Expression Language (LCEL)](https://python.langchain.com/v0.2/docs/concepts/#langchain-expression-language-lcel) 將 LangChain 模組串聯起來的簡單示例。這種方法有多種優點，包括優化的流式處理和追踪支持。

如果我們查看 LangSmith 的追踪結果，可以看到所有三個組件都出現在 [LangSmith trace](https://smith.langchain.com/public/bc49bec0-6b13-4726-967f-dbd3448b786d/r) 中。

[Serving with LangServe](https://python.langchain.com/v0.2/docs/tutorials/llm_chain/#serving-with-langserve "Direct link to Serving with LangServe")
-----------------------------------------------------------------------------------------------------------------------------------------------------

現在我們已經構建了應用程序，接下來需要將其部署。這時 LangServe 就派上用場了。LangServe 幫助開發者將 LangChain 的鏈部署為 REST API。雖然你不需要使用 LangServe 來運行 LangChain，但在本指南中，我們將展示如何使用 LangServe 部署你的應用程序。

指南的前半部分是為在 Jupyter Notebook 或腳本中運行而設計的，而現在我們將脫離這些環境。我們將創建一個 Python 文件，然後從命令行與它進行交互。

使用以下命令安裝：

```
pip install "langserve[all]"
```

### 伺服器 [Server](https://python.langchain.com/v0.2/docs/tutorials/llm_chain/#server "Direct link to Server")

要為我們的應用程序創建一個伺服器，我們需要創建一個 `serve.py` 文件。這個文件將包含我們服務應用程序的邏輯。它由三個部分組成：

1.  我們剛剛構建的鏈的定義
2.  我們的 FastAPI 應用
3.  定義一個用於提供該鏈的路由，這可以通過 `langserve.add_routes` 完成

就是這樣！如果我們執行這個文件：

```
python serve.py
```

我們應該會看到鏈條被部署在 [http://localhost:8000](http://localhost:8000/)。

### [Playground](https://python.langchain.com/v0.2/docs/tutorials/llm_chain/#playground "Direct link to Playground")

每個 LangServe 服務都附帶了一個簡單的 [內建 UI](https://github.com/langchain-ai/langserve/blob/main/README.md#playground)，可以用來配置和調用應用程序，並提供流式輸出和中間步驟的可見性。訪問 <http://localhost:8000/chain/playground/> 試試看！傳入之前相同的輸入——`{"language": "italian", "text": "hi"}`——應該會得到相同的回應。

### [Client](https://python.langchain.com/v0.2/docs/tutorials/llm_chain/#client "Direct link to Client")

現在讓我們設置一個客戶端來以程式化方式與我們的服務交互。我們可以使用 [langserve.RemoteRunnable](https://python.langchain.com/v0.2/docs/langserve/#client) 來輕鬆完成這一操作。使用這個方法，我們可以像在客戶端運行一樣與部署的鏈條交互。


In [ ]:
from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8000/chain/")
remote_chain.invoke({"language": "italian", "text": "hi"})

輸出將會是：

```
'Ciao'
```


想了解更多關於 LangServe 的其他功能，可以 [點擊這裡](https://python.langchain.com/v0.2/docs/langserve/)。

### [結論](https://python.langchain.com/v0.2/docs/tutorials/llm_chain/#conclusion "Direct link to Conclusion")
-----------------------------------------------------------------------------------------------------------------

就這樣！在這個教程中，你學會了如何創建你的第一個簡單的 LLM 應用程序。你學會了如何使用語言模型，如何解析它們的輸出，如何創建提示模板，如何使用 LCEL 將它們串聯起來，如何使用 LangSmith 提供出色的可觀測性，以及如何使用 LangServe 部署它們。

這只是成為一名熟練的 AI 工程師所需學習的冰山一角。幸運的是，我們有很多其他資源！

要進一步閱讀 LangChain 的核心概念，我們有詳細的 [概念指南](https://python.langchain.com/v0.2/docs/concepts/)。

如果你對這些概念有更具體的問題，可以查看以下部分的操作指南：

-   [LangChain Expression Language (LCEL)](https://python.langchain.com/v0.2/docs/how_to/#langchain-expression-language-lcel)
-   [Prompt templates](https://python.langchain.com/v0.2/docs/how_to/#prompt-templates)
-   [Chat models](https://python.langchain.com/v0.2/docs/how_to/#chat-models)
-   [Output parsers](https://python.langchain.com/v0.2/docs/how_to/#output-parsers)
-   [LangServe](https://python.langchain.com/v0.2/docs/langserve/)

以及 LangSmith 的文檔：

-   [LangSmith](https://docs.smith.langchain.com/)